In [3]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [4]:
from warcio.capture_http import capture_http
import requests  # requests must be imported after capture_http

In [8]:
r = requests.get("https://www.eyedrum.org/calendar-events-performances-art-music/simone-baron-arco-belo-at-eyedrum")
f = open('test.html', 'w+')
f.write(r.text)
f.close()

In [4]:
with capture_http('test.warc.gz'):
    r = requests.get("https://fingerprinter-production.herokuapp.com/visitors/9da0eb5b2cac163039f1a7900f452aae/audience_segments")


In [11]:
from warcio.archiveiterator import ArchiveIterator

with open('synchronization/downloads/nevada.egrantsmanagement.com_1.warc', 'rb') as stream:
# with open('erma.wacz', 'rb') as stream:
    for record in ArchiveIterator(stream):
        if record.rec_type == 'response':
            print(record.rec_headers.get_header('WARC-Target-URI'))

https://nevada.egrantsmanagement.com/Images/NV_New_Logo.png
https://nevada.egrantsmanagement.com/ScriptResource.axd?d=NJmAwtEo3Ipnlaxl6CMhvpHELhqt1ER2HyuSgnfGpbcF3xuclrSoif6e30TxqlI5bj1SXkLrE7PldaxKXNhtY30Mp0GXl_DvlWgpz_SAEkQC1sz0r0QjkowoI-VAkkMsRcty6FkU1NIGh1dyt324hW_qveOSqurQA-Qw7xnuUGo1&t=49337fe8
https://nevada.egrantsmanagement.com/ScriptResource.axd?d=dwY9oWetJoJoVpgL6Zq8OFjjEQM2TdpLKAgxxe-TRerL5gMtDOr8bG_bhm7586WaZTG0zRdCRk6eFdqP5FXh2OWc4tlsR4n3ymKlg2yiq6Slc0gflYuI1iJOaISCsp7o9te2sZ-1qoDtR1KKHWnDoyf4q4a4VJeeLDYzlxQZVFg1&t=49337fe8
https://nevada.egrantsmanagement.com/ScriptResource.axd?d=zvkqIRNUspAvS1yKeFhMbxcZGUwYem7Iv1beimQMBZGxV1atB_zKjSrEI46FSpZwlmS4k0kN_-uTlKa_15PjbziZQEC_jRA_PunUuBVtz52u8vFf94M4vXvlcN_RfnZHbgDWRqaXBZb4UuMTQSTZ8Q2&t=ffffffff866f772c
https://nevada.egrantsmanagement.com/WebResource.axd?d=YAYach_zykzn7tRotFpEUiM4McBsh8TAwOMojhVNpAMT4XY6tOFWyLR6nc3ba2wp_ZmwVF1AF-9QsNpbzI5oiJlsqasYiEltDVF6k5LrOyY1&t=637814660020000000
https://nevada.egrantsmanagement.com/WebRe